In [1]:
import cv2
import numpy as np
import pyautogui

# Initialize the webcam
cap = cv2.VideoCapture(0)

# Set the dimensions of the screen
screen_width, screen_height = pyautogui.size()

# Load the hand cascade for detection
hand_cascade = cv2.CascadeClassifier(r"C:\Users\PRAJES DAS\OneDrive\Desktop\4th Year Project\human activity tracker\haarcascade_hand.xml")

# Parameters for click, scroll, and mouse movement actions
prev_x, prev_y = 0, 0
scrolling = False

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        continue

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect hands in the frame
    hands = hand_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(50, 50))

    if len(hands) > 0:
        hx, hy, hw, hh = hands[0]  # Assuming one hand is detected
        hand_roi = gray[hy:hy+hh, hx:hx+hw]

        # Apply thresholding to isolate the hand
        _, hand_thresh = cv2.threshold(hand_roi, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Find contours in the thresholded image
        contours, _ = cv2.findContours(hand_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            max_contour = max(contours, key=cv2.contourArea)
            hull = cv2.convexHull(max_contour, returnPoints=False)
            hull = np.sort(hull, axis=0)  # Fix convexity defect issue
            
            if len(hull) > 3:  # Convexity defects require at least 3 points
                defects = cv2.convexityDefects(max_contour, hull)
                finger_count = 0

                if defects is not None:
                    for i in range(defects.shape[0]):
                        s, e, f, d = defects[i, 0]
                        start = tuple(max_contour[s][0])
                        end = tuple(max_contour[e][0])
                        far = tuple(max_contour[f][0])

                        a = np.linalg.norm(np.array(far) - np.array(start))
                        b = np.linalg.norm(np.array(far) - np.array(end))
                        c = np.linalg.norm(np.array(start) - np.array(end))
                        angle = np.arccos((b**2 + c**2 - a**2) / (2 * b * c))

                        if angle < np.pi / 2:
                            finger_count += 1

                # Perform actions based on finger count
                if finger_count == 1:
                    pyautogui.scroll(10)  # Scroll up
                elif finger_count == 2:
                    pyautogui.scroll(-10)  # Scroll down
                elif finger_count == 3:
                    # Move mouse based on hand movement
                    center_x = hx + hw // 2
                    center_y = hy + hh // 2
                    if prev_x and prev_y:
                        dx = center_x - prev_x
                        dy = center_y - prev_y
                        pyautogui.moveRel(dx, dy)
                    prev_x, prev_y = center_x, center_y
                elif finger_count == 0:
                    pyautogui.click()  # Click when fist is detected

    # Display the frame
    cv2.imshow('Hand Gesture Tracking', frame)

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()